The dynamic nature of Python causes surprising behaviors in its runtime performance.
Operations you might assume are slow are actually very fast (string manipulation,
generators). Language features you might assume are fast are actually very slow (attribute
access, function calls). The true source of slowdowns in a Python program can be obscure.

The best approach is to ignore your intuition and directly measure the performance of a
program before you try to optimize it. Python provides a built-in profiler for determining
which parts of a program are responsible for its execution time. This lets you focus your
optimization efforts on the biggest sources of trouble and ignore parts of the program that
don’t impact speed.

For example, say you want to determine why an algorithm in your program is slow. Here,
I define a function that sorts a list of data using an insertion sort:

The core mechanism of the insertion sort is the function that finds the insertion point for
each piece of data. Here, I define an extremely inefficient version of the insert_value
function that does a linear scan over the input array:
    
To profile insertion_sort and insert_value, I create a data set of random
numbers and define a test function to pass to the profiler.

Python provides two built-in profilers, one that is pure Python (profile) and another
that is a C-extension module (cProfile). The cProfile built-in module is better
because of its minimal impact on the performance of your program while it’s being
profiled. The pure-Python alternative imposes a high overhead that will skew the results.


### Note
When profiling a Python program, be sure that what you’re measuring is the code
itself and not any external systems. Beware of functions that access the network or
resources on disk. These may appear to have a large impact on your program’s
execution time because of the slowness of the underlying systems. If your program
uses a cache to mask the latency of slow resources like these, you should also
ensure that it’s properly warmed up before you start profiling.

Here, I instantiate a Profile object from the cProfile module and run the test
function through it using the runcall method:

    profiler = Profile()
    profiler.runcall(test)
    
Once the test function has finished running, I can extract statistics about its performance
using the pstats built-in module and its Stats class. Various methods on a Stats
object adjust how to select and sort the profiling information to show only the things you
care about.

    stats = Stats(profiler)
    stats.strip_dirs()
    stats.sort_stats(‘cumulative’)
    stats.print_stats()
    
The output is a table of information organized by function. The data sample is taken only
from the time the profiler was active, during the runcall method above.

Here’s a quick guide to what the profiler statistics columns mean:
 - ncalls: The number of calls to the function during the profiling period.
 - tottime: The number of seconds spent executing the function, excluding time spent executing other functions it calls.
 - tottime percall: The average number of seconds spent in the function each time it was called, excluding time spent executing other functions it calls. This is tottime divided by ncalls.
 - cumtime: The cumulative number of seconds spent executing the function, including time spent in all other functions it calls.
 - cumtime percall: The average number of seconds spent in the function each time it was called, including time spent in all other functions it calls. This is cumtime divided by ncalls.
 
Looking at the profiler statistics table above, I can see that the biggest use of CPU in my
test is the cumulative time spent in the insert_value function. Here, I redefine that
function to use the bisect built-in module (see Item 46: “Use Built-in Algorithms and
Data Structures”):

I can run the profiler again and generate a new table of profiler statistics. The new
function is much faster, with a cumulative time spent that is nearly 100× smaller than the
previous insert_value function.

Sometimes, when you’re profiling an entire program, you’ll find that a common utility
function is responsible for the majority of execution time. The default output from the
profiler makes this situation difficult to understand because it doesn’t show how the utility
function is called by many different parts of your program.
For example, here the my_utility function is called repeatedly by two different
functions in the program:

Profiling this code and using the default print_stats output will generate output
statistics that are confusing.

The my_utility function is clearly the source of most execution time, but it’s not
immediately obvious why that function is called so much. If you search through the
program’s code, you’ll find multiple call sites for my_utility and still be confused.
To deal with this, the Python profiler provides a way of seeing which callers contributed to
the profiling information of each function.

stats.print_callers()

This profiler statistics table shows functions called on the left and who was responsible for
making the call on the right. Here, it’s clear that my_utility is most used by
first_func:


* It’s important to profile Python programs before optimizing because the source of slowdowns is often obscure.

* Use the cProfile module instead of the profile module because it provides more accurate profiling information.

* The Profile object’s runcall method provides everything you need to profile a tree of function calls in isolation.

* The Stats object lets you select and print the subset of profiling information you need to see to understand your program’s performance.
